# Homework4

In [45]:
# import libraries
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,homogeneity_score, completeness_score
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine

## Problem 1

In [46]:
# import dataset
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
df = pd.read_csv(url, delim_whitespace=True, names=column_names, na_values='?')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1564\2313689614.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(url, delim_whitespace=True, names=column_names, na_values='?')


In [47]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car_name      398 non-null    object 
dtypes: float64(5), int64(3), object(1)
memory usage: 28.1+ KB


In [49]:
# Use only the continuous fields as features
continuous_cols = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']
df_cont = df[continuous_cols]

In [50]:
df_cont.head()

,mpg,displacement,horsepower,weight,acceleration
0,18.0,307.0,130.0,3504.0,12.0
1,15.0,350.0,165.0,3693.0,11.5
2,18.0,318.0,150.0,3436.0,11.0
3,16.0,304.0,150.0,3433.0,12.0
4,17.0,302.0,140.0,3449.0,10.5


In [51]:
df_cont.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   displacement  398 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        398 non-null    float64
 4   acceleration  398 non-null    float64
dtypes: float64(5)
memory usage: 15.7 KB


In [52]:
# impute missing values with mean
imputer = SimpleImputer(strategy='mean')
df_cont_imputed = pd.DataFrame(imputer.fit_transform(df_cont), columns=continuous_cols)

In [53]:
# standardize data (recommended for clustering)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cont_imputed)

In [54]:
# perform Hierarchical Clustering
clustering = AgglomerativeClustering(n_clusters=3, linkage='average', metric='euclidean')
cluster_labels = clustering.fit_predict(df_scaled)
df['hierarchical_cluster'] = cluster_labels

In [55]:
# Cluster statistics
print("Hierarchical Cluster Stats:")
print(df.groupby('hierarchical_cluster')[continuous_cols].agg(['mean', 'var']))

print("\nOrigin Class Stats:")
print(df.groupby('origin')[continuous_cols].agg(['mean', 'var']))

Hierarchical Cluster Stats:
                            mpg            displacement               \
                           mean        var         mean          var   
hierarchical_cluster                                                   
0                     26.177441  41.303375   144.304714  3511.485383   
1                     14.528866   4.771033   348.020619  2089.499570   
2                     43.700000   0.300000    91.750000    12.250000   

                      horsepower                   weight                 \
                            mean         var         mean            var   
hierarchical_cluster                                                       
0                      86.120275  294.554450  2598.414141  299118.709664   
1                     161.804124  674.075816  4143.969072  193847.051117   
2                      49.000000    4.000000  2133.750000   21672.916667   

                     acceleration            
                             mean   

In [56]:
# compare cluster with origin label
print("Hierarchical vs Origin:\n", pd.crosstab(df['origin'], df['hierarchical_cluster']))

Hierarchical vs Origin:
 hierarchical_cluster    0   1  2
origin                          
1                     152  97  0
2                      66   0  4
3                      79   0  0


## Problem 2

In [57]:
# import dataset
df = pd.read_csv('Boston.csv')

In [58]:
df.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [59]:
# delete Unnamed: 0
df = df.drop(columns=['Unnamed: 0'])

In [60]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [61]:
# standardize the features
X = df.select_dtypes(include='number')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [62]:
# Silhouette method
sil = []

# Perform a K-Means analysis, k ranging from 2 to 6
for k in range(2, 7):
    kmeans = KMeans(n_clusters = k).fit(X_scaled)
    labels = kmeans.labels_
    score = silhouette_score(X_scaled, labels, metric = 'euclidean')
    sil.append(score)
    print(f"k = {k}, Silhouette Score = {score:.4f}")

k = 2, Silhouette Score = 0.3501
k = 3, Silhouette Score = 0.2344
k = 4, Silhouette Score = 0.2416
k = 5, Silhouette Score = 0.1921
k = 6, Silhouette Score = 0.2789


D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known

In [63]:
# choose k=2
best_k = 2
kmeans = KMeans(n_clusters=best_k, random_state=0)
df['kmeans_cluster'] = kmeans.fit_predict(X_scaled)

D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [64]:
# mean values of all features in each cluster for the optimal clustering
cluster_means = df.groupby('kmeans_cluster').mean()
print("Mean values of all features in each cluster for the optimal clustering:")
print(cluster_means)

Mean values of all features in each cluster for the optimal clustering:
                    crim         zn      indus      chas       nox        rm  \
kmeans_cluster                                                                 
0               0.263946  17.477204   6.919818  0.069909  0.487215  6.456544   
1               9.839575   0.000000  18.975085  0.067797  0.680124  5.965096   

                      age       dis        rad         tax    ptratio  \
kmeans_cluster                                                          
0               56.382067  4.751124   4.474164  302.209726  17.818237   
1               91.238418  2.017920  18.983051  605.316384  19.640113   

                         b      lstat       medv  
kmeans_cluster                                    
0               386.643891   9.417812  25.782067  
1               300.967345  18.666610  16.493220  


In [65]:
# the centroid coordinates in scaled space
centroids = pd.DataFrame(
    kmeans.cluster_centers_,
    columns=X.columns
)
print("The centroid coordinateMean values in each cluster:")
print(centroids)

The centroid coordinateMean values in each cluster:
       crim        zn     indus      chas       nox        rm       age  \
0 -0.389801  0.262392 -0.615294  0.002912 -0.582916  0.244913 -0.433584   
1  0.724546 -0.487722  1.143682 -0.005412  1.083499 -0.455233  0.805928   

        dis       rad       tax   ptratio         b     lstat      medv  
0  0.454491 -0.583452 -0.629727 -0.294662  0.328600 -0.453497  0.353641  
1 -0.844789  1.084495  1.170509  0.547705 -0.610788  0.842942 -0.657333  


In [66]:
# compare the means and the centroid coordinates in scaled space
cluster_means.index = [f'cluster{i}_mean' for i in range(kmeans.n_clusters)]
centroids.index = [f'cluster{i}_centroid' for i in range(kmeans.n_clusters)]
comparison_df = pd.concat([cluster_means, centroids])
print("Mean values of all features in each cluster vs. Centroid coordinates (in scaled space):")
print(comparison_df)

Mean values of all features in each cluster vs. Centroid coordinates (in scaled space):
                       crim         zn      indus      chas       nox  \
cluster0_mean      0.263946  17.477204   6.919818  0.069909  0.487215   
cluster1_mean      9.839575   0.000000  18.975085  0.067797  0.680124   
cluster0_centroid -0.389801   0.262392  -0.615294  0.002912 -0.582916   
cluster1_centroid  0.724546  -0.487722   1.143682 -0.005412  1.083499   

                         rm        age       dis        rad         tax  \
cluster0_mean      6.456544  56.382067  4.751124   4.474164  302.209726   
cluster1_mean      5.965096  91.238418  2.017920  18.983051  605.316384   
cluster0_centroid  0.244913  -0.433584  0.454491  -0.583452   -0.629727   
cluster1_centroid -0.455233   0.805928 -0.844789   1.084495    1.170509   

                     ptratio           b      lstat       medv  
cluster0_mean      17.818237  386.643891   9.417812  25.782067  
cluster1_mean      19.640113  300.967345

In [67]:
# the centroid coordinates in original cluster
centroids_original = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=X.columns
)
print("The centroid coordinateMean values (in original space) in each cluster:")
print(centroids_original)

The centroid coordinateMean values (in original space) in each cluster:
       crim            zn      indus      chas       nox        rm        age  \
0  0.263946  1.747720e+01   6.919818  0.069909  0.487215  6.456544  56.382067   
1  9.839575  1.243450e-14  18.975085  0.067797  0.680124  5.965096  91.238418   

        dis        rad         tax    ptratio           b      lstat  \
0  4.751124   4.474164  302.209726  17.818237  386.643891   9.417812   
1  2.017920  18.983051  605.316384  19.640113  300.967345  18.666610   

        medv  
0  25.782067  
1  16.493220  


In [68]:
# compare the means and the centroid coordinates in original cluster
cluster_means.index = [f'cluster{i}_mean' for i in range(kmeans.n_clusters)]
centroids_original.index = [f'cluster{i}_centroid' for i in range(kmeans.n_clusters)]
comparison_df = pd.concat([cluster_means, centroids_original])
print("Mean values of all features in each cluster vs. Centroid coordinates (in original space):")
print(comparison_df)

# calculate the difference between the means and the centroid coordinates
difference_df = cluster_means.values - centroids_original.values
difference_df = pd.DataFrame(difference_df, columns=cluster_means.columns)
difference_df.index = [f'diff_cluster{i}' for i in range(kmeans.n_clusters)]
print("\nAbsolute difference between cluster means and centroid coordinates:")
print(difference_df.abs())

Mean values of all features in each cluster vs. Centroid coordinates (in original space):
                       crim            zn      indus      chas       nox  \
cluster0_mean      0.263946  1.747720e+01   6.919818  0.069909  0.487215   
cluster1_mean      9.839575  0.000000e+00  18.975085  0.067797  0.680124   
cluster0_centroid  0.263946  1.747720e+01   6.919818  0.069909  0.487215   
cluster1_centroid  9.839575  1.243450e-14  18.975085  0.067797  0.680124   

                         rm        age       dis        rad         tax  \
cluster0_mean      6.456544  56.382067  4.751124   4.474164  302.209726   
cluster1_mean      5.965096  91.238418  2.017920  18.983051  605.316384   
cluster0_centroid  6.456544  56.382067  4.751124   4.474164  302.209726   
cluster1_centroid  5.965096  91.238418  2.017920  18.983051  605.316384   

                     ptratio           b      lstat       medv  
cluster0_mean      17.818237  386.643891   9.417812  25.782067  
cluster1_mean      19.6

## Problem 3

In [69]:
# import dataset
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

In [70]:
# standardize the numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [71]:
# Perform a K-Means analysis, k=3
kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(X_scaled)

D:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [72]:
# calculate the Homogeneity/Completeness
homogeneity = homogeneity_score(y, labels)
completeness = completeness_score(y, labels)

In [73]:
homogeneity

0.8788432003662366

In [74]:
completeness

0.8729636016078731